In [9]:
# import all necesssary libraries
import requests
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

# generate a url for job position and location
def get_url(position,location):
    position = position.replace(' ','+')
    location = location.replace(' ','+')
    template = "https://in.indeed.com/jobs?q={}&l={}"
    url = template.format(position,location)
    return url

# Combining everything into a function
# Combining everything into a function
# fetching the job information from every card
def get_jobs(card):
    details = ''
    info = []
    try:
        job_name = card.find('h2','jobTitle css-4ukq4m eu4oa1w0').text
    except:
        job_name = 'Not Mentioned'
    try:
        company_name = card.find('span',"css-1x7z1ps eu4oa1w0").text
    except:
        company_name = 'Not Mentioned'
    try:
        company_loc = card.find('div',"company_location css-104iw4m e37uo190").text
    except:
        company_loc = 'Not Mentioned'
    try:
        job_salary = card.find('div','css-1ihavw2 eu4oa1w0').text
    except:
        job_salary = 'Not Mentioned'
    try:
        job_details = card.find_all('div',"metadata css-5zy3wz eu4oa1w0")
        for i in job_details:
            info.append(i.text)
        details = ",".join(str(element) for element in info)
    except:
        details = 'Not Mentioned'
    try:
        job_summary = card.find('div',"css-146u2z1 eu4oa1w0").text.strip()
    except:
        job_summary = 'Not Mentioned'
    try:
        job_link = card.find('a',"css-1f8zkg3 e19afand0")
        job_link = 'https://in.indeed.com'+ job_link['href']
    except:
        job_url = 'Not Mentioned'
        
    job_info = (job_name, company_name, company_loc, job_salary, details, job_summary, job_link)
    
    return job_info
 

def main(position,location):
    records = []
    url = get_url(position,location)
    
    driver = webdriver.Firefox()
    driver.get(url)
    driver.maximize_window()
    time.sleep(10)
    
    
    # Creating Beautifulsoup object
    soup = BeautifulSoup(driver.page_source,'html.parser')
    cards = soup.find_all('li','css-5lfssm eu4oa1w0')
    
    for everycard in cards:
        jobDetails = get_jobs(everycard)
        records.append(jobDetails)
        
    # Here we are using pandas data frame to save jobs in a csv file
    
    col = ['job_name', 'company_name', 'company_loc', 'job_salary', 'details', 'job_summary', 'job_link']
    

    indeed_jobs_data = pd.DataFrame(records,columns=col)
    indeed_jobs_data.to_csv('E:\\scrapedData\\indeed_jobs_data.csv',encoding='utf-8')

In [10]:
main('python','mumbai')